In [1]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import math
from sklearn import tree
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVR
from sklearn.svm import SVC
# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
# Cost Functions
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Preprocessing
from sklearn.impute import SimpleImputer

# Scalers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

# Feature Engineering
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
# This is for regression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# This is for classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("Consumer goods.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(exclude='number')

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(df,hue='Type')

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
#Histogram chart
plt.figure(figsize=(10,8))
sns.histplot(data=df, x='Type')
plt.title("Count of stores based on their type")
plt.ylabel("Count of stores")
plt.xlabel("Types")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.kdeplot(data=df['Price per unit'], shade=True)
sns.kdeplot(data=df['Cost per unit'], shade=True)
plt.title("Difference in price and cost per unit")
plt.xlabel("amount")
plt.tight_layout()

In [ ]:
df.isna().sum()

In [ ]:
df['Summary Profits'].fillna(df['Summary Profits'].mode()[0], inplace = True)

In [ ]:
df=df.drop(['Year','Code'],axis=1)
df.head()

In [ ]:
df['Total Cost'] = df['Total Cost'].str.replace('$','')
df['Total Cost'] = df['Total Cost'].str.replace(',','')
df['Total Cost'] = df['Total Cost'].astype(float)

In [ ]:
df['Revenues'] = df['Revenues'].str.replace('$','')
df['Revenues'] = df['Revenues'].str.replace(',','')
df['Revenues'] = df['Revenues'].astype(float)

In [ ]:
# Splitting data

# Split into training and test sets
training, test = train_test_split(
    df, 
    train_size=0.8, # 80% of data to train
    test_size=0.2, # 20% of data to test
    random_state=42 # so that regardless of how many times I run the code I will have the "exact" points
)

In [ ]:
# Create our One Hot Encoder object
one_hot = OneHotEncoder()

col_names = ["Type", "Product group","Producer"]

# One Hot encode the column in both the train and validation sets
one_hot_df = one_hot.fit_transform(training[col_names]).toarray()

one_hot_df_val = one_hot.transform(test[col_names]).toarray()

In [ ]:
# Look at the categories
one_hot.categories_

In [ ]:
# Create column names list for one hot encoded features
column_names = []

for y in range(len(one_hot.categories_)):
    for z in range(len(one_hot.categories_[y])):
        # print(one_hot.categories_[y][z])
        column_names.append(col_names[y]+"_"+one_hot.categories_[y][z])


column_names

In [ ]:
oh_df = pd.DataFrame(
    one_hot_df,
    index=training.index,
    columns = column_names
)

oh_df_val = pd.DataFrame(
    one_hot_df_val,
    index=test.index,
    columns = column_names
)

print(oh_df.shape)
print(oh_df_val.shape)

In [ ]:
# Create train_new df by merging train and one hot encoded df together and drop color and clarity cols

training = training.merge(oh_df, on=training.index).set_index("key_0").drop(["Type", "Product group","Producer"], axis=1)
test = test.merge(oh_df_val, on=test.index).set_index("key_0").drop(["Type", "Product group","Producer"], axis=1)

print(training.shape)
print(test.shape)

In [ ]:
training.head()

In [ ]:
test.head()

In [ ]:
ordinal = OrdinalEncoder()

col_names = ["Summary Profits"]

# Ordinal encode the column
ordinal_ls = ordinal.fit_transform(training[col_names])
ordinal_ls_val = ordinal.transform(test[col_names])

In [ ]:
# Look at categories
ordinal.categories_

In [ ]:
# Add oridnally encoded column to the data
training["Summary Profits"] = ordinal_ls
test["Summary Profits"] = ordinal_ls_val

training.sample(5)